In [1]:
import torch
import random
import numpy as np

SEED = 1988

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
init_token_id = tokenizer.cls_token_id
eos_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
unk_token_id = tokenizer.unk_token_id

In [5]:
max_length_input = tokenizer.max_model_input_sizes['bert-base-uncased']

In [6]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_length_input - 2]
    return tokens

In [7]:
from torchtext.data import Field, LabelField

TEXT  = Field(batch_first=True,
              use_vocab=False,
              tokenize=tokenize_and_cut,
              preprocessing=tokenizer.convert_tokens_to_ids,
              init_token=init_token_id,
              eos_token=eos_token_id,
              pad_token=pad_token_id,
              unk_token=unk_token_id)

LABEL = LabelField(dtype=torch.float)

/home/nlp/miniconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/nlp/miniconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [8]:
from torchtext import datasets

train, test = datasets.IMDB.splits(TEXT, LABEL)

/home/nlp/miniconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [9]:
LABEL.build_vocab(train)

# single GPU

In [10]:
from torchtext.data import BucketIterator

BATCH_SIZE = 32 # out of memory if batch size set to 64

#######################################################################
##################### only use one device: 'cuda:0' ###################
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#######################################################################

train_iter, test_iter = BucketIterator.splits(
    (train, test),
    batch_size=BATCH_SIZE,
    device=device
)

/home/nlp/miniconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [12]:
from torch import nn

class BertGRU(nn.Module):
    def __init__(self, bert, hidden_dim, n_layers, bidirectional, dropout):
        super().__init__()

        self.bert = bert

        embed_dim = bert.config.to_dict()['hidden_size']

        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, 
                          batch_first=True, dropout=0 if n_layers < 2 else dropout)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text): # text: [BATCH_SIZE, SEQ_LENGTH]
        with torch.no_grad():
            embedded = self.bert(text)[0] # embedded: [BATCH_SIZE, SEQ_LENGTH, EMBED_DIM]

        _, hidden = self.gru(embedded) # hidden: [N_LAYERS * n_driections, BATCH_SIZE, EMBED_DIM]

        if self.gru.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        else:
            hidden = self.dropout(hidden[-1, :, :])

        output = self.fc(hidden) # hidden: [BATCH_SIZE, 1]

        return output

In [13]:
HIDDEN_DIM = 768
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BertGRU(bert, HIDDEN_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [14]:
for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False

In [15]:
from torch import optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

In [16]:
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [18]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        preds = model(batch.text).squeeze(1)
        loss = criterion(preds.squeeze(), batch.label)
        acc = binary_accuracy(preds.squeeze(), batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            preds = model(batch.text).squeeze(1)
            loss = criterion(preds.squeeze(), batch.label)
            acc = binary_accuracy(preds.squeeze(), batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

In [21]:
N_EPOCHS = 10

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'Bert-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {test_loss:.3f} |  Val. Acc: {test_acc*100:.2f}%')

/home/nlp/miniconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Epoch Time: 26m 39s
	Train Loss: 0.371 | Train Acc: 83.24%
	 Val. Loss: 0.218 |  Val. Acc: 91.84%
Epoch: 02 | Epoch Time: 26m 44s
	Train Loss: 0.262 | Train Acc: 89.68%
	 Val. Loss: 0.224 |  Val. Acc: 91.49%
Epoch: 03 | Epoch Time: 26m 44s
	Train Loss: 0.234 | Train Acc: 90.80%
	 Val. Loss: 0.195 |  Val. Acc: 92.65%
Epoch: 04 | Epoch Time: 26m 44s
	Train Loss: 0.213 | Train Acc: 91.77%
	 Val. Loss: 0.432 |  Val. Acc: 86.01%
Epoch: 05 | Epoch Time: 26m 46s
	Train Loss: 0.212 | Train Acc: 92.05%
	 Val. Loss: 0.194 |  Val. Acc: 92.64%
Epoch: 06 | Epoch Time: 26m 44s
	Train Loss: 0.188 | Train Acc: 93.00%
	 Val. Loss: 0.198 |  Val. Acc: 92.47%
Epoch: 07 | Epoch Time: 26m 46s
	Train Loss: 0.168 | Train Acc: 93.83%
	 Val. Loss: 0.189 |  Val. Acc: 92.97%
Epoch: 08 | Epoch Time: 26m 45s
	Train Loss: 0.161 | Train Acc: 94.15%
	 Val. Loss: 0.208 |  Val. Acc: 92.44%
Epoch: 09 | Epoch Time: 26m 44s
	Train Loss: 0.163 | Train Acc: 94.02%
	 Val. Loss: 0.201 |  Val. Acc: 92.73%
Epoch: 10 